In [27]:
# MFD-2D Serial
code = """
# X-Direction
#flux step 1
fx1 := [N]->{fx1[c,y,x] : 0<=c<4 and 0<=x<=N and 0<=y<N};

#flux step 2
fx2 := [N]->{fx2[c,y,x] : 0<=c<4 and 0<=x<=N and 0<=y<N};

# Diff step 3
dx := [N]->{dx[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<N};

# Y-Direction
#flux step 1
fy1 := [N]->{fy1[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<=N};

#flux step 2
fy2 := [N]->{fy2[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<=N};

# Diff step 3
dy := [N]->{dy[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<N};

# Statements
#statement(Fx1) := "Cx1[c,y,x] = (1./12.)*(Bin[c,y,x-2] + 7.0 * Bin[c,y,x-1] +
#                                          Bin[c,y,x] + Bin[c,y,x+1])";
#statement(Fx2) := "Cx2[c,y,x] = Cx1[c,y,x] * 2.0 * Cx1[2,y,x]";
#statement(Dx) := "Bout[c,y,x] += Cx2[c,y,+1] - Cx2[c,y,x]";
#statement(Fy1) := "Cy1[c,y,x] = (1./12.)*(Bin[c,y-2,x] + 7.0 * Bin[c,y-1,x] +
#                                          Bin[c,y,x] + Bin[c,y+1,x])";
#statement(Fy2) := "Cy2[c,y,x] = Cy1[c,y,x] * 2.0 * Cy1[3,y,x]";
#statement(Dy) := "Bout[c,y,x] += Cy2[c,y+1,x] - Cy2[c,y,x]";

D := fx1 + fx2 + dx + fy1 + fy2 + dy;

S := [N] -> {
    fx1[c,y,x] -> [0,c,y,x];
    fx2[c,y,x] -> [1,c,y,x];
    dx[c,y,x] -> [2,c,y,x];
    fy1[c,y,x] -> [3,c,y,x];
    fy2[c,y,x] -> [4,c,y,x];
    dy[c,y,x] -> [5,c,y,x];
};

R := [N] -> {
    fx1[c,y,x] -> Bin[c,y,x-2];
    fx1[c,y,x] -> Bin[c,y,x-1];
    fx1[c,y,x] -> Bin[c,y,x];
    fx1[c,y,x] -> Bin[c,y,x+1];
    
    fx2[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx1[2,y,x];
    
    dx[c,y,x] -> Cx2[c,y,x];
    dx[c,y,x] -> Cx2[2,y,x+1];
    
    fy1[c,y,x] -> Bin[c,y-2,x];
    fy1[c,y,x] -> Bin[c,y-1,x];
    fy1[c,y,x] -> Bin[c,y,x];
    fy1[c,y,x] -> Bin[c,y+1,x];
    
    fy2[c,y,x] -> Cy1[c,y,x];
    fy2[c,y,x] -> Cy1[3,y,x];
    
    dy[c,y,x] -> Cy2[c,y,x];
    dy[c,y,x] -> Cy2[2,y+1,x];
} * D;

W := [N] -> {
    fx1[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx2[c,y,x];
    
    fy1[c,y,x] -> Cy1[c,y,x];    
    fy2[c,y,x] -> Cy2[c,y,x];
    
    dx[c,y,x] -> Bout[c,y,x];    
    dy[c,y,x] -> Bout[c,y,x];
} * D;

codegen(S * D);

#B4 := S << S;
#RaW := (W . (R^-1)) * B4;
#print "RaW deps:";
#print RaW;
#WaW := (W . (W^-1)) * B4;
#print "WaW deps:";
#print WaW;
#WaR := (R . (W^-1)) * B4;
#print "WaR deps:";
#print WaR;
"""

from tools.system import run
run('/usr/local/bin/iscc', code, verbose=True)


Output:

{
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 <= N; c3 += 1)
        fx1(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 <= N; c3 += 1)
        fx2(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        dx(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 <= N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        fy1(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 <= N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        fy2(c1, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        dy(c1, c2, c3);
}



('{\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 <= N; c3 += 1)\n        fx1(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 <= N; c3 += 1)\n        fx2(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        dx(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 <= N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        fy1(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 <= N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        fy2(c1, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        dy(c1, c2, c3);\n}\n',
 '')

In [1]:
# MFD-2D -- Slice & Reschedule (iteration space split)
code = """
# slice(Fx1, c=2)
Fx1 := [N]->{fx1[c,y,x]: 0<=c<4 and 0<=y<N and 0<=x<=N};
Fx2 := [N]->{fx2[c,y,x] : 0<=c<4 and 0<=x<=N and 0<=y<N};
Dx := [N]->{dx[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<N};

# slice(Fy1, c=3)
Fy1 := [N]->{fy1[c,y,x]: 0<=c<4 and 0<=y<=N and 0<=x<N};
Fy2 := [N]->{fy2[c,y,x] : 0<=c<4 and 0<=y<=N and 0<=x<N};
Dy := [N]->{dy[c,y,x] : 0<=c<4 and 0<=y<N and 0<=x<N};

# Statements
#statement(Fx1) := "Cx1[c,y,x] = (1./12.)*(Bin[c,y,x-2] + 7.0 * Bin[c,y,x-1] +
#                                          Bin[c,y,x] + Bin[c,y,x+1])";
#statement(Fx2) := "Cx2[c,y,x] = Cx1[c,y,x] * 2.0 * Cx1[2,y,x]";
#statement(Dx) := "Bout[c,y,x] += Cx2[c,y,+1] - Cx2[c,y,x]";
#statement(Fy1) := "Cy1[c,y,x] = (1./12.)*(Bin[c,y-2,x] + 7.0 * Bin[c,y-1,x] +
#                                          Bin[c,y,x] + Bin[c,y+1,x])";
#statement(Fy2) := "Cy2[c,y,x] = Cy1[c,y,x] * 2.0 * Cy1[3,y,x]";
#statement(Dy) := "Bout[c,y,x] += Cy2[c,y+1,x] - Cy2[c,y,x]";

D := Fx1 + Fx2 + Dx + Fy1 + Fy2 + Dy;

# shift(dx, x, 1)
# shift(dy, y, 1)
# slice(Fx1, c=2)
# slice(Fy1, c=3)

S := [N] -> {
    fx1[c,y,x] -> [0,c,y,x,0]: c=2;
    fy1[c,y,x] -> [0,c,y,x,0]: c=3;
    fx1[c,y,x] -> [1,c,y,x,0]: 0<=c<=1;
    fx1[c,y,x] -> [1,c,y,x,0]: c=3;
    fx2[c,y,x] -> [1,c,y,x,1];
    dx[c,y,x] -> [1,c,y,x+1,2];
    fy1[c,y,x] -> [2,c,y,x,0]: 0<=c<=2;
    fy2[c,y,x] -> [2,c,y,x,1];
    dy[c,y,x] -> [2,c,y+1,x,2];
};

R := [N] -> {
    fx1[c,y,x] -> Bin[c,y,x-2];
    fx1[c,y,x] -> Bin[c,y,x-1];
    fx1[c,y,x] -> Bin[c,y,x];
    fx1[c,y,x] -> Bin[c,y,x+1];
    
    fx2[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx1[2,y,x];
    
    dx[c,y,x] -> Cx2[c,y,x];
    dx[c,y,x] -> Cx2[2,y,x+1];
    
    fy1[c,y,x] -> Bin[c,y-2,x];
    fy1[c,y,x] -> Bin[c,y-1,x];
    fy1[c,y,x] -> Bin[c,y,x];
    fy1[c,y,x] -> Bin[c,y+1,x];
    
    fy2[c,y,x] -> Cy1[c,y,x];
    fy2[c,y,x] -> Cy1[3,y,x];
    
    dy[c,y,x] -> Cy2[c,y,x];
    dy[c,y,x] -> Cy2[2,y+1,x];
} * D;

W := [N] -> {
    fx1[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx2[c,y,x];
    
    fy1[c,y,x] -> Cy1[c,y,x];    
    fy2[c,y,x] -> Cy2[c,y,x];
    
    dx[c,y,x] -> Bout[c,y,x];    
    dy[c,y,x] -> Bout[c,y,x];
} * D;

codegen(S * D);
"""

from tools.system import run
run('/usr/local/bin/iscc', code, verbose=True)


Output:

for (int c0 = 0; c0 <= 2; c0 += 1) {
  if (c0 == 2) {
    for (int c1 = 0; c1 <= 3; c1 += 1)
      for (int c2 = 0; c2 <= N; c2 += 1)
        for (int c3 = 0; c3 < N; c3 += 1) {
          if (c1 <= 2)
            fy1(c1, c2, c3);
          fy2(c1, c2, c3);
          if (c2 >= 1)
            dy(c1, c2 - 1, c3);
        }
  } else if (c0 == 1) {
    for (int c1 = 0; c1 <= 3; c1 += 1)
      for (int c2 = 0; c2 < N; c2 += 1)
        for (int c3 = 0; c3 <= N; c3 += 1) {
          if (c1 <= 1) {
            fx1(c1, c2, c3);
          } else if (c1 == 3) {
            fx1(3, c2, c3);
          }
          fx2(c1, c2, c3);
          if (c3 >= 1)
            dx(c1, c2, c3 - 1);
        }
  } else {
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 <= N; c3 += 1)
        fx1(2, c2, c3);
    for (int c2 = 0; c2 <= N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1)
        fy1(3, c2, c3);
  }
}



('for (int c0 = 0; c0 <= 2; c0 += 1) {\n  if (c0 == 2) {\n    for (int c1 = 0; c1 <= 3; c1 += 1)\n      for (int c2 = 0; c2 <= N; c2 += 1)\n        for (int c3 = 0; c3 < N; c3 += 1) {\n          if (c1 <= 2)\n            fy1(c1, c2, c3);\n          fy2(c1, c2, c3);\n          if (c2 >= 1)\n            dy(c1, c2 - 1, c3);\n        }\n  } else if (c0 == 1) {\n    for (int c1 = 0; c1 <= 3; c1 += 1)\n      for (int c2 = 0; c2 < N; c2 += 1)\n        for (int c3 = 0; c3 <= N; c3 += 1) {\n          if (c1 <= 1) {\n            fx1(c1, c2, c3);\n          } else if (c1 == 3) {\n            fx1(3, c2, c3);\n          }\n          fx2(c1, c2, c3);\n          if (c3 >= 1)\n            dx(c1, c2, c3 - 1);\n        }\n  } else {\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 <= N; c3 += 1)\n        fx1(2, c2, c3);\n    for (int c2 = 0; c2 <= N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1)\n        fy1(3, c2, c3);\n  }\n}\n',
 '')

In [24]:
# MFD-2D -- Fusion Within
code = """
# slice(Fx1, c=2)
Fx1 := [N]->{fx1_0[c,y,x]: 0<=y<N and 0<=x<=N and 0<=c<=1;
             fx1_1[c,y,x]: 0<=y<N and 0<=x<=N and c=2;
             fx1_2[c,y,x]: 0<=y<N and 0<=x<=N and c=3};
Fx2 := [N]->{fx2[c,y,x] : 0<=c<4 and 0<=x<=N and 0<=y<N};
Dx := [N]->{dx[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<N};

# slice(Fy1, c=3)
Fy1 := [N]->{fy1_0[c,y,x]: 0<=y<=N and 0<=x<N and 0<=c<=2;
             fy1_1[c,y,x]: 0<=y<=N and 0<=x<N and c=3};
Fy2 := [N]->{fy2[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<=N};
Dy := [N]->{dy[c,y,x] : 0<=c<4 and 0<=x<N and 0<=y<N};


# Statements
#stmt(Fx1) := "Cx1[c,y,x] = (1./12.)*(Bin[c,y,x-2] + 7.0 * Bin[c,y,x-1] +
#                                          Bin[c,y,x] + Bin[c,y,x+1])";
#stmt(Fx2) := "Cx2[c,y,x] = Cx1[c,y,x] * 2.0 * Cx1[2,y,x]";
#stmt(Dx) := "Bout[c,y,x] += Cx2[c,y,+1] - Cx2[c,y,x]";
#stmt(Fy1) := "Cy1[c,y,x] = (1./12.)*(Bin[c,y-2,x] + 7.0 * Bin[c,y-1,x] +
#                                          Bin[c,y,x] + Bin[c,y+1,x])";
#stmt(Fy2) := "Cy2[c,y,x] = Cy1[c,y,x] * 2.0 * Cy1[3,y,x]";
#stmt(Dy) := "Bout[c,y,x] += Cy2[c,y+1,x] - Cy2[c,y,x]";

D := Fx1 + Fx2 + Dx + Fy1 + Fy2 + Dy;

# fuse(fx1_1, fy1_1);
# shift(dx, x, 1)
# shift(dy, y, 1)
# fuse(fx1_0, fx1_2, fx2, dx);
# fuse(fy1_0, fy1_1, fy2, dy);

S := [N] -> {
    fx1_1[c,y,x] -> [0,c,y,x,0];
    fy1_1[c,y,x] -> [0,c,y,x,0];
    fx1_0[c,y,x] -> [1,c,y,x,0];
    fx1_2[c,y,x] -> [1,c,y,x,0];
    fx2[c,y,x] -> [1,c,y,x,1];
    dx[c,y,x] -> [1,c,y,x+1,2];
    fy1_0[c,y,x] -> [2,c,y,x,0];
    fy2[c,y,x] -> [2,c,y,x,1];
    dy[c,y,x] -> [2,c,y+1,x,2];
};

R := [N] -> {
    fx1[c,y,x] -> Bin[c,y,x-2];
    fx1[c,y,x] -> Bin[c,y,x-1];
    fx1[c,y,x] -> Bin[c,y,x];
    fx1[c,y,x] -> Bin[c,y,x+1];
    
    fx2[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx1[2,y,x];
    
    dx[c,y,x] -> Cx2[c,y,x];
    dx[c,y,x] -> Cx2[2,y,x+1];
    
    fy1[c,y,x] -> Bin[c,y-2,x];
    fy1[c,y,x] -> Bin[c,y-1,x];
    fy1[c,y,x] -> Bin[c,y,x];
    fy1[c,y,x] -> Bin[c,y+1,x];
    
    fy2[c,y,x] -> Cy1[c,y,x];
    fy2[c,y,x] -> Cy1[3,y,x];
    
    dy[c,y,x] -> Cy2[c,y,x];
    dy[c,y,x] -> Cy2[2,y+1,x];
} * D;

W := [N] -> {
    fx1[c,y,x] -> Cx1[c,y,x];
    fx2[c,y,x] -> Cx2[c,y,x];
    
    fy1[c,y,x] -> Cy1[c,y,x];    
    fy2[c,y,x] -> Cy2[c,y,x];
    
    dx[c,y,x] -> Bout[c,y,x];    
    dy[c,y,x] -> Bout[c,y,x];
} * D;

codegen(S * D);
"""

# Is this legal? With a little unpeeling, these loops are now fusable...
#for (int c2 = 0; c2 < N; c2 += 1)
#  for (int c3 = 0; c3 < N; c3 += 1) {
#    fx1_1(2, c2, c3);
#    fy1_1(3, c2, c3);  
#  }    
#  fx1_1(2, c2, N);      
#  for (int c3 = 0; c3 < N; c3 += 1)
#      fy1_1(3, N, c3);       

from tools.system import run
run('/usr/local/bin/iscc', code, verbose=True)


Output:

{
  for (int c2 = 0; c2 < N; c2 += 1)
    for (int c3 = 0; c3 <= N; c3 += 1)
      fx1_1(2, c2, c3);
  for (int c2 = 0; c2 <= N; c2 += 1)
    for (int c3 = 0; c3 < N; c3 += 1)
      fy1_1(3, c2, c3);
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 < N; c2 += 1)
      for (int c3 = 0; c3 <= N; c3 += 1) {
        if (c1 <= 1) {
          fx1_0(c1, c2, c3);
        } else if (c1 == 3) {
          fx1_2(3, c2, c3);
        }
        fx2(c1, c2, c3);
        if (c3 >= 1)
          dx(c1, c2, c3 - 1);
      }
  for (int c1 = 0; c1 <= 3; c1 += 1)
    for (int c2 = 0; c2 <= N; c2 += 1)
      for (int c3 = 0; c3 < N; c3 += 1) {
        if (c1 <= 2)
          fy1_0(c1, c2, c3);
        fy2(c1, c2, c3);
        if (c2 >= 1)
          dy(c1, c2 - 1, c3);
      }
}



('{\n  for (int c2 = 0; c2 < N; c2 += 1)\n    for (int c3 = 0; c3 <= N; c3 += 1)\n      fx1_1(2, c2, c3);\n  for (int c2 = 0; c2 <= N; c2 += 1)\n    for (int c3 = 0; c3 < N; c3 += 1)\n      fy1_1(3, c2, c3);\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 < N; c2 += 1)\n      for (int c3 = 0; c3 <= N; c3 += 1) {\n        if (c1 <= 1) {\n          fx1_0(c1, c2, c3);\n        } else if (c1 == 3) {\n          fx1_2(3, c2, c3);\n        }\n        fx2(c1, c2, c3);\n        if (c3 >= 1)\n          dx(c1, c2, c3 - 1);\n      }\n  for (int c1 = 0; c1 <= 3; c1 += 1)\n    for (int c2 = 0; c2 <= N; c2 += 1)\n      for (int c3 = 0; c3 < N; c3 += 1) {\n        if (c1 <= 2)\n          fy1_0(c1, c2, c3);\n        fy2(c1, c2, c3);\n        if (c2 >= 1)\n          dy(c1, c2 - 1, c3);\n      }\n}\n',
 '')